# Bibliographic Networks: A Python Tutorial

Networks can provide significant measures to identify data driven patterns and dependencies. Though, given a data file it can be difficult to discern how one may approach creating such a network. In this tutorial, we will use a bibliographic data file downloaded from a query search in [Scopus](https://https://www.scopus.com/search/form.uri) to walk through the process of cleaning the data file, writing a python script to parse the data into nodes and edges, computing graphical measures using [NetworkX](https://https://networkx.github.io/documentation/stable/index.html), and creating an interactive network display using [HoloViews](https://http://holoviews.org). 

### 1. Data Manipulation in Excel

As you are editing and cleaning your data set, be sure to always save in Excel as <i>CSV UTF-8 (Comma delimited) (.csv)</i>. This will ensure that the data file is readable by the Python reader used in this tutorial, and will keep any special characters. 

#### Scopus Specific Data Manipulation 
Few Scopus downloadable queries are perfect. This tutorial uses the Scopus file containing results for the query <i>economics AND "complex systems."</i> Upon downloading this data file, some rows are skewed from inaccurate reading and parsing. If you are costumizing this tutorial, simply scroll through the file and delete any rows where the data is clearly mismatched (i.e. an author name in the 'Title' column, a numerical value in a non-numerical column, etc.). 

Additionally, across several different queries, we discovered duplicates in entry 'Title,' with other columns containing conflicting data. To fix this issue for the purposes of producing a network, duplicates should be removed. With your .csv file open in Excel, select <i>Data -> Table Tools -> Remove Duplicates</i>. Indicate that the .csv file has headers, as all Scopus files will, and only select the 'Title' column by which duplicates will be identified. After executing this command, it is important to again save the file as a .csv. Otherwise, Excel may default to saving the file as a .txt, or another format, and data features may be lost. By continuously saving the file as a .csv, we ensure that it will continue to be compatible with the Python code for this tutorial.

Generally, for the case of creating a connected network, we want the rows in our bibliographic data file to have a unique title and a list of references. Other customizations can be made as long as this feature is preserved. 

### 2. Import Necessary Libraries and Packages 
The following code will download the necessary libraries and packages for this tutorial. To customize this tutorial, decalare your own .csv file. 

In [ ]:
!pip install networkx -q
!pip install numpy -q
!pip install pandas -q
!pip install holoviews -q
!pip install bokeh -q
!pip install scikit-image -q
!pip install xarray -q
!pip install datashader -q

In [8]:
import csv
import networkx as nx
import numpy as np
import pandas as pd
import holoviews as hv
from holoviews.operation.datashader import datashade, bundle_graph
from networkx.algorithms import community

file_name = 'scopus.csv' # TODO: insert filename if customizing this tutorial

ImportError: No module named networkx

### 3. Partitioning the Data into Nodes and Edges

This Python script is specific to SCOPUS and bibliometric data, though could be easily customized to match the parameters of any data file. 

To make a network, we must identify objects and relationships between objects. With bibliometric data, we can identify titles and designate a connection between titles if one is referenceing the other. The downloaded Scopus file identifies a title for a source in each row. The column 'References' indicates a semicolon delimited list of references in MLA or APA format. To make this information useful, we must parse an identifying title from each reference in the list. Because the Scopus file does not maintain consistent formatting accross references, we simply keep the entire reference and search for strings inside strings, though this could be customized depending on data file formatting. 

In [ ]:
node_list = [] # a list of titles and references
edge_list = [] # includes rows of format [a, b] where 'a' references 'b'
type_dict = {} # key: node, value: type ('title' or 'reference'), holds all possible node values

''' 
Requires: 'n_type' is either 'title' or 'reference' 
Modifies: If 'node' occurs in the list, preserves type 'title,' changing either 
          the 'node_list' value and the 'type_dict' type, or just the 'node' value.
          Else, adds 'node' to 'node_list.'
Effects:  Compares 'node' to the current 'note_list.' 
'''
def comp_add(node_list, node, n_type):
    for i in range(len(node_list)): 
        # check to see if 'node' compares to any current nodes
        if node in node_list[i] or node_list[i] in node: 
            # if a node exists as a row 'title' and a row 'reference', 
            # we want to favor the type 'title' in our data structures 
            if n_type == 'title': 
                # switch the representation in 'node_list' to 'title'
                node_list[i] = node
                type_dict[node] = n_type 
            else:
                # switch the representation of 'node' to 'title' 
                node = node_list[i]
            return node 
        
    # the rest of this function executes if 'node' is not already in 'node_list'
    if n_type == 'title':
        node_list.append(node)
        type_dict[node] = n_type
    else: 
        node_list.append(node)
        type_dict[node] = n_type

    return node

''' 
Main loop to parse data into nodes and edges. 
'''
with open(file_name) as csv_file:
    reader = csv.DictReader(csv_file)
    for row in reader:
        # add node with unique identifier
        source_node = row['Title']
        source_node = comp_add(node_list, source_node, 'title')
        # add an edge for each source and its references
        refs = row['References'].split(';')
        for ref in refs:
            # disregard web references, and clean data for any formatting inconsistencies
            if 'https://' not in ref and 'http://' not in ref and ref != " " and ref != "":  
                ref = comp_add(node_list, ref, 'reference')
                edge = [source_node, ref] # 'source_node' references 'ref'
                edge_list.append(edge) 


### 4. Graph Manipulation 
Once you have created an <i>edge_list</i> variable, edges can be added to a NetworkX graph. Using NetworkX for this graph manipulation is intuitive and clean, requiring minimal lines of code. We use a directed graph so that we can determine the frequency by which a node is being referenced, as opposed to the frequency by which a node is referencing. 

In [ ]:
G = nx.DiGraph()
for n in node_list: 
    G.add_node(n)
G.add_edges_from(edge_list)

For a large graph, depending on the information being represented, one may want to prune the graph to only contain nodes with a degree (the number of connections to a single node) greater than 1. For this bibliometric data, we are primarily interested in the connections between nodes, therefore a node with only one connection is of much less importance. Furthermore, by removing less significant nodes, we can decrease the graph size significantly, creating a more easily understood graphical layout. Be careful to run this code only as many times as you wish to reduce the graph, or else significant information may be lost as the graph is pruned, depending on the degree of interest in the information.  

In [ ]:
# by running this code once, all isolated subgraphs will be removed 

# first remove nodes of degree 1
nodes_to_remove = []
for n in G.nodes(): 
    if G.degree(n) == 1: 
        nodes_to_remove.append(n)
G.remove_nodes_from(nodes_to_remove)

# then remove nodes that are isolated 
nodes_to_remove = []
for n in G.nodes(): 
    if G.degree(n) == 0: 
        nodes_to_remove.append(n)
G.remove_nodes_from(nodes_to_remove)

Now that we have pruned our graph, to give the nodes a distinguishable measure, we indicate a label for each node corresponding to its type, and a measure of degree. This part could be customized to distinguish a node by any measure. By adding attributes in this way, they will carry over to HoloViews.  

In [ ]:
for n in G.nodes:
    G.node[n]['label'] = type_dict[n]
    G.node[n]['in-degree'] = G.in_degree(n)

Next we will run the [NetworkX Girvan Newman] (https://networkx.github.io/documentation/latest/reference/algorithms/generated/networkx.algorithms.community.centrality.girvan_newman.html) algorithm on the graph to identify community structure. 

In [ ]:
communities_generator = community.girvan_newman(G)
communities = next(communities_generator)

# sort the communities based on size 
groups = []
for c in communities: 
    size = len(c)
    groups.append([size, list(c)])
groups.sort(reverse=True)

# give each node a community id to which it belongs 
# groups with a lower 'group_id' will be a member of a larger group
group_id = 1
for group in groups:
    members = group[1]
    for n in members: 
        G.node[n]['community'] = group_id
    group_id += 1

### 5. Creating a Graphical Display
See inline comments for any places for further customization. A brief discussion of specific functions and layouts are provided in the GitHub README. 

##### Functionality for the Following Display 
Nodes are sized according to their in-degree, a measure of the number of sources that are referencing the node. Colors are assigned according to the NetworkX Girvan Newman community identification algorithm. Edges are bundled to create a clear flow of connection, as opposed to a cluttered, overlapping display. Hover over an edge to identify the end nodes of an edge. End nodes will be highlighted in green when hovering over an edge.       

In [ ]:
hv.extension('bokeh')
kwargs = dict(width=1000, height=1000, xaxis=None, yaxis=None)
hv.opts.defaults(hv.opts.Nodes(**kwargs), hv.opts.Graph(**kwargs)) 

# choose a NetworkX graphical layout 
pos = nx.spring_layout(G,k=0.15, iterations=20) 
# collect graph from NetworkX 
my_graph = hv.Graph.from_networkx(G, pos)
# bundle edges 
bundled = bundle_graph(my_graph)
bundled.opts(padding=0.1) 
# add community and in-degree features
bundled.opts(node_color=hv.dim('community'), node_size=((hv.dim('in-degree') + 1)*5), width=1000, 
                          cmap='Colorblind')
bundled.opts(inspection_policy='edges')